In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import lxml

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from pandas import json_normalize
import json


In [40]:
url = 'https://fbref.com/en/matches/a843d023/Arsenal-Brighton-and-Hove-Albion-August-31-2024-Premier-League'
page = requests.get(url)
soup = BeautifulSoup(page.text,'lxml')
scorebox = soup.find('div',{'class':"scorebox"})
scorebox.find_all('div',{'class':'media-item logo loader'})

table = soup.find_all('table')

#Shots table: Both squads
shots_table1 = table[17]

# Extract column headers
# The first row of headers contains merged columns, so we'll get headers from the second row
headers = [th.get_text(strip=True) for th in shots_table1.find_all('tr')[1].find_all('th')]

# Initialize list to hold row data
rows_data = []

# Extract data from each row
for tr in shots_table1.find_all('tbody')[0].find_all('tr'):
    cols = tr.find_all(['th', 'td'])
    row_data = [col.get_text(strip=True) for col in cols]
    rows_data.append(row_data)

# Create the DataFrame
df = pd.DataFrame(rows_data, columns=headers)
df

,Minute,Player,Squad,xG,PSxG,Outcome,Distance,Body Part,Notes,Player,Event,Player,Event
0,2,Bukayo Saka,Arsenal,0.06,0.23,Saved,3,Left Foot,,Bukayo Saka,Take-On,Kai Havertz,Pass (Live)
1,8,Bukayo Saka,Arsenal,0.11,0.10,Saved,9,Left Foot,,Martin Ødegaard,Pass (Live),Bukayo Saka,Pass (Live)
2,11,Ben White,Arsenal,0.04,0.00,Blocked,16,Right Foot,,Bukayo Saka,Take-On,William Saliba,Pass (Live)
3,14,Yankuba Minteh,Brighton,0.02,0.00,Saved off Target,12,Head,,João Pedro,Pass (Live),Kaoru Mitoma,Pass (Live)
4,18,Martin Ødegaard,Arsenal,0.18,0.10,Saved,11,Right Foot,Volley,Leandro Trossard,Pass (Live),Ben White,Pass (Live)
5,28,Jack Hinshelwood,Brighton,0.03,0.00,Blocked,31,Right Foot,,Lewis Dunk,Pass (Live),Carlos Baleba,Pass (Live)
6,31,Kaoru Mitoma,Brighton,0.04,0.00,Off Target,16,Left Foot,Volley,Carlos Baleba,Pass (Live),,
7,38,Kai Havertz,Arsenal,0.57,0.25,Goal,25,Left Foot,,Bukayo Saka,Pass (Live),Jurriën Timber,Pass (Live)
8,45+2,Leandro Trossard,Arsenal,0.03,0.20,Saved,18,Right Foot,,Gabriel Magalhães,Pass (Live),William Saliba,Pass (Live)
9,,,,,,,,,,,,,


In [41]:
import html
import re
import pandas as pd

# Find all event elements using regex to match class names starting with 'event'
events = soup.find_all('div', class_=re.compile(r'^event\s'))

# Initialize lists to hold event data
times = []
scores = []
players = []
event_types = []
teams = []

# Extract data for each event
for event in events:
    # Extract time and score
    time_score_div = event.find('div')
    if time_score_div:
        time_score_text = html.unescape(time_score_div.get_text(strip=True))
        time = time_score_text.split("'")[0] + "'" if "'" in time_score_text else time_score_text
        score = time_score_text.split("'")[1] if "'" in time_score_text else ''
        times.append(time)
        scores.append(score)

    # Extract player name
    player = event.find('a').get_text(strip=True) if event.find('a') else ''
    players.append(player)
    
    # Extract event type
    event_type = 'Unknown'
    for div in event.find_all('div'):
        if '—' in div.get_text():
            event_type = div.get_text(strip=True).split('—')[-1].strip()
            break
    event_types.append(event_type)
    
    # Extract team
    team_logo = event.find('img', class_='teamlogo')
    if team_logo:
        team_name = team_logo.get('alt').replace(' Club Crest', '')
        teams.append(team_name)
    else:
        teams.append('Unknown')

# Create a DataFrame
red_cards_df = pd.DataFrame({
    'Time': times,
    'Score': scores,
    'Player': players,
    'Event Type': event_types,
    'Team': teams
})

# Filter for both types of red cards
red_cards_df = red_cards_df[red_cards_df['Event Type'].isin(['Red Card', 'Second Yellow Card'])]

# Reset index to make it clean
red_cards_df = red_cards_df.reset_index(drop=True)

red_cards_df

,Time,Score,Player,Event Type,Team
0,49’1:0,,Declan Rice,Second Yellow Card,Arsenal


In [42]:
red_cards_df["Minute"] = red_cards_df["Time"].str[:2]
red_cards_df["Outcome"] = "Red Card"

df["Minute"] = df["Minute"].str[:2]
df["Team"] = df["Squad"]
df = df[["Minute", "Team", "xG", "PSxG", "Outcome"]]

df["Event Type"] = "Shot"

df.drop(df[df['Minute'] == ''].index, inplace=True)

red_cards_df = red_cards_df[["Minute", "Team", "Event Type", "Outcome"]]

df = pd.concat([df, red_cards_df], ignore_index=True)

df["Minute"] = df["Minute"].astype(int)
df["xG"] = df["xG"].astype(float)
df["PSxG"] = df["PSxG"].astype(float)
df.fillna(0.00, inplace=True)
df.sort_values(by=["Minute"], inplace=True)

df

C:\Users\Owner\AppData\Local\Temp\ipykernel_20604\2867144827.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Event Type"] = "Shot"
C:\Users\Owner\AppData\Local\Temp\ipykernel_20604\2867144827.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['Minute'] == ''].index, inplace=True)


,Minute,Team,xG,PSxG,Outcome,Event Type
0,2,Arsenal,0.06,0.23,Saved,Shot
1,8,Arsenal,0.11,0.10,Saved,Shot
2,11,Arsenal,0.04,0.00,Blocked,Shot
3,14,Brighton,0.02,0.00,Saved off Target,Shot
4,18,Arsenal,0.18,0.10,Saved,Shot
5,28,Brighton,0.03,0.00,Blocked,Shot
6,31,Brighton,0.04,0.00,Off Target,Shot
7,38,Arsenal,0.57,0.25,Goal,Shot
8,45,Arsenal,0.03,0.20,Saved,Shot
9,46,Arsenal,0.03,0.00,Off Target,Shot


In [ ]:
red_card_matched = ["https://fbref.com/en/matches/a843d023/Arsenal-Brighton-and-Hove-Albion-August-31-2024-Premier-League",
                    ""]